In [1]:
import json
import numpy as np
import pandas as pd
import psycopg2
import os

from collections import Counter
from datetime import datetime, timedelta
from dateutil.tz import tzlocal 
from pathlib import Path
from tqdm import tqdm

## Analyse data

In [2]:
from concurrent.futures import ThreadPoolExecutor

def load_responses(resp_path):
    resp_tm = datetime.strptime(resp_path.name[6:-5], '%Y-%m-%d %H;%M;%S')
    resp_tm_str = resp_tm.replace(tzinfo=tzlocal()).isoformat()
    
    with open(resp_path, 'r', encoding='utf-8' ) as f:
        resp = json.load(f)
        if ("rows" in resp) and (resp["rows"]): 
            return [row + [resp['timestamp'], resp_tm_str] 
                    for row in resp['rows']]
        else: 
            return []

def accumulate_responses_from_folder(folder_path):
    file_path_list = list(folder_path.iterdir())
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(tqdm(executor.map(load_responses, file_path_list),
                            total=len(file_path_list), mininterval=10, leave=False, 
                            desc="Accumulate responses from folder")
                          )
        resp_list = []
        for resp_ in results: 
            resp_list += resp_
        print(f"Accumulate responses from folder\n"\
              f"\t{len(resp_list)} / {len(resp_list)} [avg={len(resp_list)/len(resp_list):.2f}]")
    
    
    columns = ['imei', 'lat', 'lng', 'speed', 'gps_datetime_origin', 'orientation', 'route_name', 
               'route_type', 'vehicle_id', 'dd', 'gpstime', 'response_datetime'] 

    return pd.DataFrame(resp_list, columns=columns)
    
def clear_data(in_df):
    unique_data = []
    
    imei_list = in_df['imei'].value_counts().index
    imei_tqdm = tqdm(
        imei_list, total=len(imei_list), mininterval=10, leave=False, desc="Clear data"
    )
    
    for imei in imei_tqdm: 
        row_data = in_df[ in_df['imei'] == imei].values.tolist()
        result = [row_data[0]]
        for row0, row1 in zip(row_data[:-1], row_data[1:]):
            if row0[1:9] != row1[1:9]: 
                result += [row1]
        unique_data += result

    df_unique = pd.DataFrame(unique_data, columns=in_df.columns)
    print(f"Clear data\n\t{len(in_df)} / {len(df_unique)} [avg {len(in_df) / len(df_unique):.02f}]")
    return df_unique

In [15]:
"kh_requests_path".upper()

'KH_REQUESTS_PATH'

In [7]:
KHARKIV_FOLDER_PATH = Path("../../data/local/kharkiv")
KH_REQUESTS_PATH = KHARKIV_FOLDER_PATH/"jsons"
KH_TABLES_PATH = KHARKIV_FOLDER_PATH/"tables"
# kharkiv_folder_path = Path("../../../pet_project/kharkiv")
kharkiv_req_list = [p for p in KH_REQUESTS_PATH.iterdir() if "trans_data_" in p.name]
kharkiv_reqs_list = sorted(kharkiv_folder_list,
                        key=lambda p: datetime.strptime(p.name[11:], '%d_%b_%Y'))
# kharkiv_folders_list

for folder_path in tqdm(kharkiv_reqs_list[:-1]):   
    print(f"Processing '{folder_path}'")
    df = accumulate_responses_from_folder(folder_path)
    df.to_parquet(KH_TABLES_PATH/'origin'/(folder_path.name + '_origin.parquet'))
    
    df_u = clear_data(df)
    df_u.to_parquet(KH_TABLES_PATH/'optimized'/(folder_path.name + '_optimized.parquet'))

  0%|          | 0/24 [00:00<?, ?it/s]

Processing '..\..\data\local\kharkiv\jsons\trans_data_17_MAR_2023'



Accumulate responses from folder:  89%|████████▉ | 15463/17280 [00:33<00:03, 459.39it/s]
                                                                                        

Accumulate responses from folder
	2469322 / 2469322 [avg=1.00]



Clear data:  83%|████████▎ | 230/278 [00:20<00:04, 11.64it/s]
                                                             

...

Clear data
	2139128 / 81808 [avg 26.15]
Processing '..\..\data\local\kharkiv\jsons\trans_data_09_APR_2023'



Accumulate responses from folder:  91%|█████████ | 15737/17280 [00:50<00:05, 299.92it/s]
                                                                                        

Accumulate responses from folder
	2104581 / 2104581 [avg=1.00]



100%|██████████| 24/24 [39:21<00:00, 98.41s/it]    

Clear data
	2104581 / 78996 [avg 26.64]


In [20]:
# [p.name[11:-18] for p in file_path_list]

In [23]:
file_path_list = list((KH_TABLES_PATH/"optimized").iterdir())
file_path_list = sorted(file_path_list,
                        key=lambda p: datetime.strptime(p.name[11:-18], '%d_%b_%Y'))
file_path_list[-3:]

[WindowsPath('../../data/local/kharkiv/tables/optimized/trans_data_11_APR_2023_optimized.parquet'),
 WindowsPath('../../data/local/kharkiv/tables/optimized/trans_data_12_APR_2023_optimized.parquet'),
 WindowsPath('../../data/local/kharkiv/tables/optimized/trans_data_13_APR_2023_optimized.parquet')]

In [24]:
df_l = pd.read_parquet(file_path_list[-1])

In [25]:
df['route_name'].value_counts().index

Index(['3', '27', '8', '20', '204', '6', '119', '1', '63', '24', '35', '16А',
       '16', '19', '304', '272', '55', '11', '31', '45', '46', '34', '56',
       '72', '170', '7', '13', '267', '109', '57', '106', '40', '140', '32',
       '260', '5', '112', '7А', '51', '49', '222', '96', '58', '243', '29',
       '80', '67', '77', '33', '12', '81', '53', '94', '25', '68', '18', '21',
       '23', '47', '99', '64', '28', '83', '224', '241', '273', '78', '118',
       '22', '41'],
      dtype='object')

In [26]:
df_u['route_name'].value_counts().index

Index(['3', '27', '8', '20', '204', '6', '1', '119', '24', '35', '16А', '16',
       '63', '19', '304', '11', '34', '272', '31', '55', '56', '45', '7', '13',
       '170', '46', '72', '5', '7А', '40', '32', '49', '267', '112', '106',
       '140', '260', '109', '57', '51', '222', '12', '18', '29', '58', '33',
       '80', '81', '96', '94', '77', '68', '53', '243', '67', '23', '21', '83',
       '241', '47', '25', '64', '78', '28', '99', '273', '118', '224', '41',
       '22'],
      dtype='object')

In [27]:
df_l['route_name'].value_counts().index

Index(['3', '27', '20', '8', '204', '6', '35', '24', '63', '119', '1', '19',
       '272', '304', '16', '16А', '34', '45', '170', '11', '13', '31', '55',
       '7', '7А', '72', '46', '56', '112', '51', '68', '106', '58', '33', '32',
       '49', '260', '109', '57', '96', '140', '40', '77', '5', '222', '94',
       '267', '81', '25', '80', '52', '18', '29', '12', '53', '67', '83',
       '243', '241', '23', '273', '47', '78', '21', '64', '22', '41', '99',
       '28', '118', '224'],
      dtype='object')

In [28]:
df[df['route_name'] == '56А']

,imei,lat,lng,speed,gps_datetime_origin,orientation,route_name,route_type,vehicle_id,dd,gpstime,response_datetime


In [29]:
df[df['route_name'] == '56Рђ']['route_name'].astype('string')

Series([], Name: route_name, dtype: string)

In [29]:
import pyarrow.parquet as pq
pfile = pq.read_table(file_path_list[-2])

In [30]:
pfile

pyarrow.Table
imei: int64
lat: int64
lng: int64
speed: int64
gps_datetime_origin: int64
orientation: int64
route_name: string
route_type: int64
vehicle_id: int64
dd: int64
gpstime: string
response_datetime: string
----
imei: [[353976014023992,353976014023992,353976014023992,353976014023992,353976014023992,...,352625696583044,352625696583044,352625696583044,352625696583044,352625696583044],[352625696583044,352625696583044,352625696583044,352625696583044,352625696583044,...,353976014241727,353976014241727,353976014241727,353976014241727,353976014241727]]
lat: [[369808,370071,368210,367470,366741,...,237099,237099,251362,253208,253410],[259300,259159,259102,259220,259090,...,260399,260410,260448,260429,260380]]
lng: [[961990,961880,959579,958759,957932,...,984879,984879,977291,976440,976349],[970169,968700,968250,968281,967739,...,944111,944019,944099,945011,945950]]
speed: [[0,5,17,17,12,...,13,0,21,23,17],[26,27,12,0,24,...,0,0,0,28,11]]
gps_datetime_origin: [[0,0,0,0,0,...,0,0,0,0,0],[

In [27]:
import pyarrow

In [16]:
BATCH_SIZE = 3_000
connection_config= dict({
    'host': os.environ['RDS_HOSTNAME'],
    'database': "pteta_db",
    'user': "postgres",
    'password': os.environ['RDS_PTETA_DB_PASSWORD']
})

monitor = TransGPSCVMonitor(connection_config=connection_config, data_model="kharkiv")

# file_path_list =list(Path("../data/local/tables/").iterdir())
file_path_list =list(Path("D:/projects/pet_project/tables").iterdir())
file_path_list = sorted(file_path_list, 
                        key=lambda p: datetime.strptime(p.name[9:-4], '%d_%b_%Y') )

for folder_path in tqdm(kharkiv_folders_list[:-1]):
    df = accumulate_responses_from_folder(folder_path)
    df_u = clear_data(df)       
    
    df_cur = df_u[:]

    batch_tqdm = tqdm(df_cur.groupby(np.arange(len(df_cur)) // BATCH_SIZE), 
                     mininterval=10)
    for batch_number, batch_df in batch_tqdm:
        batch_df = batch_df.where(pd.notnull(batch_df), None)
        monitor.write_to_db(batch_df.to_dict('records'))

Accumulate responses from folder:  98%|█████████▊| 12441/12739 [01:28<00:01, 231.33it/s]
                                                                                        

Accumulate responses from folder
	1616588 / 1616588 [avg=1.00]



Clear data:  86%|████████▌ | 235/273 [00:19<00:03, 12.44it/s]
                                                             

Clear data
	1616588 / 153418 [avg 10.54]



Accumulate responses from folder:  99%|█████████▉| 10925/10997 [00:43<00:00, 254.63it/s]
                                                                                        

...


Clear data:  95%|█████████▌| 229/241 [00:19<00:00, 12.19it/s]
                                                             
  0%|          | 0/68 [00:00<?, ?it/s]

Clear data
	2075486 / 201332 [avg 10.31]



 37%|███▋      | 25/68 [00:20<00:34,  1.24it/s]

There are 1 new <class 'PTETA.utils.transport.kharkiv.KharkivTransportVehicle.KharkivTransportVehicle'> to inserted in DB



100%|██████████| 3/3 [06:44<00:00, 134.70s/it]


In [6]:
monitor.objects_unique

{PTETA.utils.transport.kharkiv.KharkivTransportOperator.KharkivTransportOperator: {KharkivTransportOperator(id=-1, name='UNKNOWN')},
 PTETA.utils.transport.kharkiv.KharkivTransportRoute.KharkivTransportRoute: {KharkivTransportRoute(id=-1),
  KharkivTransportRoute(id=1),
  KharkivTransportRoute(id=10),
  KharkivTransportRoute(id=11),
  KharkivTransportRoute(id=12),
  KharkivTransportRoute(id=13),
  KharkivTransportRoute(id=14),
  KharkivTransportRoute(id=15),
  KharkivTransportRoute(id=16),
  KharkivTransportRoute(id=17),
  KharkivTransportRoute(id=18),
  KharkivTransportRoute(id=19),
  KharkivTransportRoute(id=2),
  KharkivTransportRoute(id=20),
  KharkivTransportRoute(id=21),
  KharkivTransportRoute(id=22),
  KharkivTransportRoute(id=23),
  KharkivTransportRoute(id=24),
  KharkivTransportRoute(id=25),
  KharkivTransportRoute(id=5),
  KharkivTransportRoute(id=6),
  KharkivTransportRoute(id=7),
  KharkivTransportRoute(id=8),
  KharkivTransportRoute(id=9)},
 PTETA.utils.transport.kharkiv

In [7]:
operator_list, route_list, vehicle_list, avl_data_list = monitor.decompose_response(batch_df.to_dict('records'))

for obj_list in [operator_list, route_list, vehicle_list]:
    new_obj = monitor.get_new_objs(obj_list)
    if new_obj:
        print(f"There are {len(new_obj)} new {new_obj[0].__class__} to inserted in DB")
        monitor.update_db(new_obj)

for i, (vehicle, route) in enumerate(zip(vehicle_list, route_list)):
    avl_data_list[i].vehicle_id = monitor.vehicle_to_id[vehicle]
    avl_data_list[i].route_id = monitor.route_to_id[route]


There are 1 new <class 'PTETA.utils.transport.kharkiv.KharkivTransportRoute.KharkivTransportRoute'> to inserted in DB


KeyError: KharkivTransportRoute(id=None)

In [8]:
i, (vehicle, route)

(732,
 (KharkivTransportVehicle(id=None, imei='353976014249548', name='None', owner_id=-1),
  KharkivTransportRoute(id=None)))

In [9]:
route.id, route.name, route.type

(None, '27', 1)

In [ ]:
route.__

In [10]:
[(r.id, r.name, r.type) for r in monitor.objects_unique[route.__class__]]

[(8, '8', 1),
 (25, '28', 1),
 (9, '208', 3),
 (16, '3', 2),
 (23, '75', 3),
 (6, '204', 3),
 (22, '68', 3),
 (15, '206', 3),
 (14, '212', 3),
 (19, '51', 2),
 (-1, 'UNKNOWN', -1),
 (17, '1', 2),
 (21, '20', 1),
 (18, '6', 2),
 (7, '45', 2),
 (12, '35', 2),
 (11, '46', 2),
 (1, '52', 2),
 (2, '55', 2),
 (5, '24', 2),
 (20, '13', 2),
 (13, '72', 3),
 (10, '34', 2),
 (24, '27', 2)]

NameError: name 'batch_df' is not defined

In [7]:
#     df_cur = df_u[:]

batch_tqdm = tqdm(df_cur.groupby(np.arange(len(df_cur)) // BATCH_SIZE), 
                  miniters=40)
for batch_number, batch_df in batch_tqdm:
    batch_df = batch_df.where(pd.notnull(batch_df), None)
    monitor.write_to_db(batch_df.to_dict('records'))

  0%|          | 0/124 [00:00<?, ?it/s]

There are 2 new <class 'PTETA.utils.transport.kharkiv.KharkivTransportRoute.KharkivTransportRoute'> to inserted in DB
Error raised while select <class 'PTETA.utils.transport.kharkiv.KharkivTransportRoute.KharkivTransportRoute'> '[KharkivTransportRoute(id=None, name='52', type=2), KharkivTransportRoute(id=None, name='55', type=2)]'


SyntaxError: syntax error at or near "AND"
LINE 1: SELECT "name", "type" FROM kharkiv.route  WHERE ( AND "name"...
                                                          ^


In [32]:
from psycopg2.extras import RealDictCursor
connection_config= dict({
    'host': os.environ['RDS_HOSTNAME'],
    'database': "pteta_db",
    'user': "postgres",
    'password': os.environ['RDS_PTETA_DB_PASSWORD']
})
conn = psycopg2.connect(**connection_config)

# cur = connection.cursor(cursor_factory = RealDictCursor)

In [54]:
with conn.cursor() as cur : 
    cur.execute('SELECT id, "name", "type" FROM kharkiv.route;')
    columns = [desc[0] for desc in cur.description]
    real_dict = [dict(zip(columns, row)) for row in cur.fetchall()]
    print(real_dict)

[{'id': -1, 'name': 'UNKNOWN', 'type': -1}, {'id': 1, 'name': '52', 'type': 2}, {'id': 2, 'name': '55', 'type': 2}, {'id': 3, 'name': '24', 'type': 2}, {'id': 4, 'name': '204', 'type': 3}, {'id': 5, 'name': '45', 'type': 2}, {'id': 6, 'name': '8', 'type': 1}, {'id': 7, 'name': '208', 'type': 3}, {'id': 8, 'name': '34', 'type': 2}, {'id': 9, 'name': '46', 'type': 2}, {'id': 10, 'name': '35', 'type': 2}, {'id': 11, 'name': '72', 'type': 3}, {'id': 12, 'name': '212', 'type': 3}, {'id': 13, 'name': '206', 'type': 3}, {'id': 14, 'name': '3', 'type': 2}, {'id': 15, 'name': '1', 'type': 2}, {'id': 16, 'name': '6', 'type': 2}, {'id': 17, 'name': '51', 'type': 2}, {'id': 18, 'name': '13', 'type': 2}, {'id': 19, 'name': '20', 'type': 1}, {'id': 20, 'name': '68', 'type': 3}, {'id': 21, 'name': '75', 'type': 3}, {'id': 22, 'name': '27', 'type': 2}, {'id': 23, 'name': '28', 'type': 1}, {'id': 24, 'name': '27', 'type': 1}, {'id': 25, 'name': '281', 'type': 3}, {'id': 26, 'name': '119', 'type': 2}, {

In [55]:
**real_dict

SyntaxError: invalid syntax (2672971826.py, line 1)

['id', 'name']

In [13]:
cv_req_path = Path(r"D:\projects\pet_project\trans_data_23_FEB_2023")
len([p.name for p in cv_req_path.iterdir()]), [p.name for p in cv_req_path.iterdir()][30_000]

(54701, 'trans_2023-02-23 14;31;48.json')

In [14]:
with open(cv_req_path/'trans_2023-02-23 14;31;48.json', 'r', encoding='utf-8') as file:
    resp = json.load(file)

In [15]:
resp

{'355227045533718': {'id': 156,
  'imei': '355227045533718',
  'name': 'A79',
  'stateCode': 'used',
  'stateName': 'used',
  'lat': 48.26898333333333,
  'lng': 25.927051666666667,
  'speed': '022.4',
  'orientation': '220.37',
  'gpstime': '2023-02-23 13:31:46',
  'routeId': 2,
  'routeName': '11',
  'routeColour': 'green',
  'inDepo': False,
  'busNumber': '9034',
  'perevId': 1,
  'perevName': 'Денисівка',
  'remark': '9034 DNSNK',
  'online': True,
  'idBusTypes': 1},
 '355227046451662': {'id': 394,
  'imei': '355227046451662',
  'name': 'H76',
  'stateCode': 'used',
  'stateName': 'used',
  'lat': 48.292473333333334,
  'lng': 25.935496666666666,
  'speed': '000.0',
  'orientation': '089.35',
  'gpstime': '2023-02-23 13:31:46',
  'routeId': 31,
  'routeName': '6/6a',
  'routeColour': 'deeppink',
  'inDepo': False,
  'busNumber': '350',
  'perevId': 6,
  'perevName': 'ЧТУ',
  'remark': 'Тролейбус 350 DNSNTNK',
  'online': True,
  'idBusTypes': 2},
 '355228042956811': {'id': 413,
  '

In [45]:
response_datetime

odict_items([('id', -1), ('name', 'UNKNOWN')])

## TransportRoute

In [7]:
TransportRoute.__insert_columns__().replace('"', '').split(', ')[1:]

['route_name', 'route_colour']

In [28]:
cols = ["routeId", 'routeName', 'routeColour']
route_list = [TransportRoute.from_response_row(row) for row in df_sum[cols].drop_duplicates().to_dict('records')]
len(route_list), route_list[:3]
TransportRoute.insert_many_in_table(conn, route_list)

In [32]:
TransportRoute.get_table(conn)

[TransportRoute(id=37, name='T', colour='coral'),
 TransportRoute(id=31, name='6/6a', colour='deeppink'),
 TransportRoute(id=21, name='38', colour='deeppink'),
 TransportRoute(id=41, name='10A', colour='black'),
 TransportRoute(id=20, name='A', colour='navy'),
 TransportRoute(id=42, name='39', colour='coral'),
 TransportRoute(id=23, name='19', colour='teal'),
 TransportRoute(id=19, name='10', colour='black'),
 TransportRoute(id=11, name='9', colour='magenta'),
 TransportRoute(id=2, name='11', colour='green'),
 TransportRoute(id=6, name='4', colour='magenta'),
 TransportRoute(id=27, name='3/3a', colour='green'),
 TransportRoute(id=16, name='2', colour='green'),
 TransportRoute(id=4, name='5', colour='orange'),
 TransportRoute(id=3, name='12', colour='blue'),
 TransportRoute(id=12, name='20', colour='maroon'),
 TransportRoute(id=7, name='6', colour='sienna'),
 TransportRoute(id=10, name='34', colour='navy'),
 TransportRoute(id=45, name='1', colour='navy'),
 TransportRoute(id=9, name='27'

In [13]:
# df_sum[TransportRoute.__insert_columns__().replace('"', '').split(', ')[1:]].value_counts()

In [27]:
conn.rollback()

In [45]:
SQL_big_req = " ".join([create_sql_req(t) for t in trans_vehicle_list])

In [29]:
route_list = TransportRoute.get_table(conn)[:]
len(route_list)

31

In [30]:
for r in route_list[::2]: 
    r.id += 1
TransportRoute.are_in_table(conn, route_list)

[False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False]

In [31]:
TransportRoute.are_in_table(conn, route_list)

[False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False]

In [14]:
for r in route_list: 
    r.name += "_"
    r.id += 100

In [16]:
# sql = f"""INSERT INTO pteta.route("id", "routeName", "routeColour") VALUES """ + \
#               ", ".join([f"""({obj.id}, '{obj.name}', '{obj.colour}')"""
#                         for obj in route_list]) + ";"
# sql

In [17]:
# route_list

In [18]:
TransportRoute.insert_many_in_table(conn, route_list)

## Test TransportVehicle

In [33]:
TransportVehicle.__insert_columns__()

'"imei", "name", "bus_number", "remark", "perev_id"'

In [45]:
cols = ["imei", "name", "busNumber", "remark", "perevId"]
vehicle_list = [TransportVehicle.from_response_row(row) for row in df_sum[cols].drop_duplicates().to_dict('records')]
len(vehicle_list), vehicle_list[:3]
TransportVehicle.insert_many_in_table(conn, vehicle_list)

In [47]:
vehicle_list = TransportVehicle.get_table(conn)
len(vehicle_list)

99

In [48]:
TransportVehicle.are_in_table(conn, vehicle_list[:5])

[True, True, True, True, True]

In [49]:
vehicle_list[5].is_in_table(conn)

True

In [50]:
obj = vehicle_list[0]
obj.imei += "-"
obj.is_in_table(conn)

False

In [51]:
obj.insert_in_table(conn)

In [52]:
for v in vehicle_list[:10]: 
    v.imei += '_'

In [15]:
TransportVehicle.insert_many_in_table(conn, vehicle_list[:10])

In [31]:
obj.insert_in_table(conn)
obj.is_in_table(conn)

In [14]:
vehicle_list[0].id = None
print(vehicle_list[0])
vehicle_list[0].update_id_from_table(conn)
print(vehicle_list[0])

TransportVehicle(id=None, imei='355227045600830', name='A178', busNumber='310', remark='Тролейбус 310 DNSNK', perevId=6, routeId=37)
TransportVehicle(id=1, imei='355227045600830', name='A178', busNumber='310', remark='Тролейбус 310 DNSNK', perevId=6, routeId=37)


## TransportOperator

In [41]:
operator_list = TransportOperator.get_table(conn)
len(operator_list)

6

In [37]:
TransportOperator.__insert_columns__()

'id, "perev_name"'

In [40]:
cols = ['perevId', "perevName"]
operator_list = [TransportOperator.from_response_row(row) 
              for row in df_sum[cols].drop_duplicates().to_dict('records')]
len(operator_list), operator_list[:3]
TransportOperator.insert_many_in_table(conn, operator_list)

In [42]:
TransportOperator.are_in_table(conn, operator_list[:])

[True, True, True, True, True, True]

In [43]:
operator_list[0].is_in_table(conn)

True

In [44]:
obj = operator_list[0]
obj.id += 100
obj.name = "-_-" + obj.name
obj.is_in_table(conn)

False

In [14]:
obj.insert_in_table(conn)

In [15]:
for op in operator_list: 
    op.id += 100
    op.name = "-_-" + op.name
    
TransportOperator.insert_many_in_table(conn, operator_list)

## TransportAVLData

In [9]:
avl_data_list = TransportAVLData.get_table(conn)

In [11]:
# avl_data_list

In [14]:
%%timeit
TransportAVLData.are_in_table(conn, avl_data_list)

57.4 ms ± 3.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
avl_data_list[0].is_in_table(conn)

42.1 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
file_path_list[0].name[:-4]

'data_for_01_DEC_2022'

In [58]:
file_path = Path(f"../../../pet_project/tables")
file_path_list = [p for p in file_path.iterdir() if ".csv" in p.name]
(file_path/"parquet").mkdir(exist_ok=True)

# errors in 22
for folder_path in tqdm(file_path_list[:]):   
    print(f"Processing '{folder_path}'")
    df = pd.read_csv(folder_path)
    
    df["busNumber"] = df["busNumber"].astype(str)
    df.to_parquet(folder_path.parent/'parquet'/(folder_path.name[:-4] + '.parquet'))

  0%|          | 0/126 [00:00<?, ?it/s]

Processing '..\..\..\pet_project\tables\data_for_01_DEC_2022.csv'


  1%|          | 1/126 [00:00<01:44,  1.20it/s]

Processing '..\..\..\pet_project\tables\data_for_01_FEB_2023.csv'


  2%|▏         | 2/126 [00:01<01:22,  1.50it/s]

...

 98%|█████████▊| 124/126 [02:54<00:02,  1.48s/it]

Processing '..\..\..\pet_project\tables\data_for_31_JAN_2023.csv'


 99%|█████████▉| 125/126 [02:55<00:01,  1.37s/it]

Processing '..\..\..\pet_project\tables\data_for_31_OCT_2022.csv'


100%|██████████| 126/126 [02:56<00:00,  1.40s/it]


In [53]:
Path.mkdir?

Signature: Path.mkdir(self, mode=511, parents=False, exist_ok=False)
Docstring: Create a new directory at this given path.
File:      c:\program files\python38\lib\pathlib.py
Type:      function


In [ ]:
(file_path/"parquet").mkdir